In [7]:
import os
import cv2
import uuid
import numpy as np
from matplotlib import pyplot as plt

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Layer
from sklearn.model_selection import train_test_split

In [50]:
# Set these before running
DATA_DIR = r"E:\face recognition data\Class"
EMBEDDING_PATH = r"E:\face recognition data\class_embeddings.npy"
CLASS_NAMES_PATH = r"E:\face recognition data\class_names.npy"
MODEL_WEIGHTS_PATH = r"E:\face recognition data\siamese_model.h5"
MODEL_DIR = r"E:\face recognition data"

In [19]:
MODE = "train"  # change to "realtime" to run OpenCV-based identification or "data_collection" to collect images
IMAGE_SIZE = (100, 100)

In [11]:
def collect_images():
    print("\nStarting image collection... Press 's' to save, 'q' to quit.")
    name = input("Enter class name: ")
    class_path = os.path.join(DATA_DIR, name)
    os.makedirs(class_path, exist_ok=True)

    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        roi = frame[120:120+330, 150:150+330, :]
        cv2.imshow("Capturing", roi)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            imgname = os.path.join(class_path, '{}.jpg'.format(uuid.uuid1()))
            cv2.imwrite(imgname, roi)
            print(f"Saved {imgname}")
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [12]:
def preprocess_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, IMAGE_SIZE)
    img = img / 255.0
    return img

In [13]:
def create_pairs():
    pairs = []
    labels = []
    class_dirs = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
    class_to_imgs = {
        c: [os.path.join(DATA_DIR, c, f) for f in os.listdir(os.path.join(DATA_DIR, c))]
        for c in class_dirs
    }
    for c in class_dirs:
        imgs = class_to_imgs[c]
        for i in range(len(imgs)):
            for j in range(i + 1, len(imgs)):
                pairs.append((imgs[i], imgs[j]))
                labels.append(1)
            other_classes = [k for k in class_dirs if k != c]
            for neg_class in other_classes:
                neg_img = np.random.choice(class_to_imgs[neg_class])
                pairs.append((imgs[i], neg_img))
                labels.append(0)
    return pairs, labels, class_dirs

In [14]:
def make_embedding_model():
    inp = Input(shape=(100, 100, 3))
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inp)
    x = MaxPooling2D()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(128, activation='sigmoid')(x)
    return Model(inp, out)

In [15]:
class L1Dist(Layer):
    def call(self, a, b):
        return tf.math.abs(a - b)

In [16]:
def make_siamese_model(embedding):
    input_a = Input(shape=(100, 100, 3))
    input_b = Input(shape=(100, 100, 3))
    emb_a = embedding(input_a)
    emb_b = embedding(input_b)
    distance = L1Dist()(emb_a, emb_b)
    output = Dense(1, activation='sigmoid')(distance)
    return Model([input_a, input_b], output)

In [45]:
def train():
    print("[INFO] Starting training...")

    # Load image paths
    class_to_imgs = {}
    for class_dir in os.listdir(DATA_DIR):
        class_path = os.path.join(DATA_DIR, class_dir)
        if not os.path.isdir(class_path): continue
        image_files = [os.path.join(class_path, f) for f in os.listdir(class_path) if f.lower().endswith((".jpg", ".png"))]
        class_to_imgs[class_dir] = image_files

    class_names = list(class_to_imgs.keys())

    # Generate training pairs (positive and negative)
    pairs = []
    labels = []

    for class_dir in class_names:
        images = class_to_imgs[class_dir]
        for i in range(len(images) - 1):
            img1, img2 = preprocess_image(images[i]), preprocess_image(images[i + 1])
            pairs.append((img1, img2))
            labels.append(1)

            # Sample a negative from another class
            other_class = random.choice([c for c in class_names if c != class_dir])
            neg_img = preprocess_image(random.choice(class_to_imgs[other_class]))
            pairs.append((img1, neg_img))
            labels.append(0)

    # Convert to numpy arrays
    pair_left = np.array([p[0] for p in pairs])
    pair_right = np.array([p[1] for p in pairs])
    labels = np.array(labels)

    # Train the model
    embedding = make_embedding_model()
    siamese_model = make_siamese_model(embedding)

    siamese_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    siamese_model.fit([pair_left, pair_right], labels, batch_size=16, epochs=10)

    # Save models
    siamese_model.save(os.path.join(MODEL_DIR, "siamese_model.h5"))
    embedding.save_weights(os.path.join(MODEL_DIR, "embedding.weights.h5"))  # Save embedding weights
    embedding.

    # Create and save embeddings for all classes
    embedding_model = make_embedding_model()
    embedding_model.load_weights(os.path.join(MODEL_DIR, "embedding.weights.h5"))  # Correct weights

    class_embeddings = {}
    for class_dir in class_names:
        imgs = class_to_imgs[class_dir][:5]  # use first 5
        embs = [embedding_model.predict(np.expand_dims(preprocess_image(p), axis=0))[0] for p in imgs]
        mean_emb = np.mean(embs, axis=0)
        class_embeddings[class_dir] = mean_emb

    np.save(os.path.join(MODEL_DIR, "class_embeddings.npy"), class_embeddings)
    print("[INFO] Training and embedding creation complete.")


In [52]:
def realtime():
    model = tf.keras.models.load_model(MODEL_WEIGHTS_PATH, custom_objects={'L1Dist': L1Dist})
    embedding = model.get_layer(index=2).input[0]._keras_history[0]  # not elegant, works
    class_embeddings = np.load(EMBEDDING_PATH, allow_pickle=True).item()
    class_names = np.load(CLASS_NAMES_PATH)

    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        roi = frame[120:120+330, 150:150+330, :]
        img = cv2.resize(roi, IMAGE_SIZE)
        img = img / 255.0
        emb = embedding.predict(np.expand_dims(img, axis=0))[0]

        sims = [np.linalg.norm(emb - ce) for ce in class_embeddings]
        min_idx = np.argmin(sims)
        min_dist = sims[min_idx]

        name = class_names[min_idx] if min_dist < 0.6 else "Unknown"

        cv2.rectangle(frame, (150, 120), (480, 450), (0, 255, 0), 2)
        cv2.putText(frame, f"{name} ({min_dist:.2f})", (150, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.imshow("Real-time Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [46]:
if MODE == "data_collection":
    collect_images()
elif MODE == "train":
    train()
elif MODE == "realtime":
    realtime()

[INFO] Starting training...
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.6925
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - accuracy: 0.2000 - loss: 0.7036
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step - accuracy: 0.5000 - loss: 0.6807
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step - accuracy: 0.6000 - loss: 0.6821
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step - accuracy: 0.7000 - loss: 0.6465
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 0.5000 - loss: 0.6886
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step - accuracy: 0.6000 - loss: 0.6826
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 0.4000 - loss: 0.7210
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.4000 - loss: 0.7131
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.5000 - loss: 0.6689


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
[INFO] Training and embedding creation complete.


In [54]:
realtime()

AttributeError: 'GetItem' object has no attribute 'predict'